# Optimal Control

In [1]:
import os
import numpy as np
from numpy.testing import assert_array_equal, assert_array_almost_equal
import matplotlib.pyplot as plt
from casadi import (
    MX, DM, Function, Opti, sparsify,
    vec, vcat, vertcat, horzcat, blockcat, sum1, sum2, sumsqr,
    gradient, jacobian, hessian
)

In [2]:
data_dir = 'data'

## 1. Solve Multiple Shooting Problem

In [3]:
# State transition function
uk = MX.sym('uk')
xk = MX.sym('xk')
F = Function('F', [xk, uk], [xk ** 2 + uk], ['xk', 'uk'], ['xkp1'])
F

Function(F:(xk,uk)->(xkp1) MXFunction)

Discrete optimal control problem:

$$
\DeclareMathOperator*{\minimize}{minimize}
\begin{aligned}
\minimize_{x_1,x_2,...,x_{N+1},u_1,u_2,...,u_N} \quad & \sum_{k=1}^{N}{u_k^2} + \sum_{k=1}^{N+1}{x_k^2} \\
\text{subject to} \quad & F(x_k,u_k) = x_{k+1}, \quad k=1, ..., N \\
                  & x_1 = 2 \\
                  & x_{N+1} = 3 \\
                  & x_k \ge 0, \quad k=1, ..., N+1 \\
\end{aligned}
$$


In [4]:
opti_MS = Opti()
N = 4
X = opti_MS.variable(N + 1)
U = opti_MS.variable(N)

# Define objective function
f = sum1(X ** 2) + sum1(U ** 2)
# or sumsqr(X) + sumsqr(U)

# Define constraints
for k in range(1, N + 1):
    opti_MS.subject_to(X[k] == F(X[k-1], U[k-1]))
opti_MS.subject_to(X[0] == 2)  # initial constraint
opti_MS.subject_to(X[1:-1] >= 0)  # path constraint
opti_MS.subject_to(X[-1] == 3)  # final constraint

opti_MS.minimize(f)

In [5]:
opti_MS.solver('ipopt')
sol_MS = opti_MS.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       14
Number of nonzeros in inequality constraint Jacobian.:        3
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:        9
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        6
Total number of inequality c

In [6]:
sol_MS.value(U)

array([-2.70380638, -0.54297936,  0.26125189,  0.58403971])

In [7]:
assert_array_almost_equal(sol_MS.value(U), [-2.7038, -0.5430, 0.2613, 0.5840], decimal=4)

## 2. Adapt to single-shooting problem

In [8]:
opti_SS = Opti()

# Objective function
N = 4
x0 = 2  # initial state
U = opti_SS.variable(N)
X = [x0]
for k in range(1, N + 1):
    X.append(F(X[k-1], U[k-1]))
X = vertcat(*X)
f = sum1(X ** 2) + sum1(U ** 2)

# Define constraints
opti_SS.subject_to(X[1:-1] >= 0)
opti_SS.subject_to(X[-1] == 3)

opti_SS.minimize(f)

In [9]:
X[-1]

MX(F(F(F(F(2, opti1_x_1[0]){0}, opti1_x_1[1]){0}, opti1_x_1[2]){0}, opti1_x_1[3]){0})

In [10]:
opti_SS.solver('ipopt')
sol_SS = opti_SS.solve()

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        3
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.2950331e+09 6.55e+04 1.21e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [11]:
sol_SS.value(U)

array([-2.70380008, -0.54298561,  0.26123341,  0.58403084])

In [12]:
assert_array_almost_equal(sol_SS.value(U), [-2.7038, -0.5430, 0.2613, 0.5840], decimal=4)

## 3. Plot sparsities of the constraint Jacobian and Hessian

### Constraint Jacobian

In [13]:
constraint_J_SS = jacobian(opti_SS.g, opti_SS.x)
constraint_J_SS.shape

(4, 4)

In [14]:
constraint_J_SS.sparsity().spy()

*...
**..
***.
****


\begin{aligned}
x_1 \ge 0 \\
F(x_1, u_1) \ge 0 \\
F(F(x_1, u_1), u_2) \ge 0 \\
F(F(F(x_1, u_1), u_2), u_3) \ge 0 \\
\end{aligned}

In [15]:
constraint_J_MS = jacobian(opti_MS.g, opti_MS.x)
constraint_J_MS.shape

(9, 9)

In [16]:
constraint_J_MS.sparsity().spy()

**...*...
.**...*..
..**...*.
...**...*
*........
.*.......
..*......
...*.....
....*....


This is made up of both path constraint and gap-closing constraints:

\begin{aligned}
F(x_k, u_k) - x_{k+1} = 0 \\
x_k \ge 0
\end{aligned}

Analyse terms of constraint Jacobian. Let $g_k = F(x_k, u_k) - x_{k+1}$

Non-zero terms relating to gap constraints:

\begin{aligned}
\frac{\partial g_k}{\partial x_{k+1}} = -1 \\
\frac{\partial g_k}{\partial x_k} = \frac{\partial F}{\partial x_k} \\
\frac{\partial g_k}{\partial u_k} = \frac{\partial F}{\partial u_k}
\end{aligned}

Path constraints are just the diagonal elements:

\begin{aligned}
\frac{\partial g_k}{\partial x_{k}} = 1
\end{aligned}

### Hessian

In [17]:
opti_SS.f.shape, opti_SS.x.shape

((1, 1), (4, 1))

In [18]:
H_SS = hessian(opti_SS.f, opti_SS.x)[0]
H_SS.shape

(4, 4)

In [19]:
H_SS.sparsity().spy()

****
****
****
****


In single-shooting, the future states are dependent on the inputs in all previous time steps so the Hessian has many cross-terms (i.e. dense). The way the objective is defined, the relationship between f and u is highly non-linear and therefore all the terms in the Hessian are non-zero.

\begin{aligned}
x_5 \leftarrow F(F(F(x_1, u_1), u_2), u_3), u_4)
\end{aligned}

All inputs have an effect.  Everything is coupled.

$$\frac{\partial^2 x_5}{\partial(u_1, u_2, u_3, u_4)}$$

Therefore all elements are non-zero.  (Whole trajectory affects the outcome.)

In [20]:
opti_MS.f.shape, opti_MS.x.shape

((1, 1), (9, 1))

In [21]:
H_MS = hessian(opti_MS.f, opti_MS.x)[0]
H_MS.shape

(9, 9)

In [22]:
H_MS.sparsity().spy()

*........
.*.......
..*......
...*.....
....*....
.....*...
......*..
.......*.
........*


In multiple-shooting there are more decision variables but each decision variable is independent (no complex dependencies) and the objective is a sum-of-squared terms, therefore the Hessian is sparse (diagonal).

$$
F(x_k, u_k) = \sum_{k=1}^n {x_k^2} + = \sum_{k=1}^n {u_k^2}
$$

## 4. Banded constraint Jacobian

In [23]:
# To make the constraint Jacobian more banded, arrange the xk and uk terms next to each other:

opti_MS = Opti()
x0 = opti_MS.variable()
opti_MS.subject_to(x0==2)

U = []
X = [x0]
for k in range(1, N + 1):
    xkm1 = X[k-1]
    uk = opti_MS.variable()
    xk = opti_MS.variable()
    opti_MS.subject_to(F(xkm1, uk) == xk)
    if k < N:
        opti_MS.subject_to(xk >= 0)
    U.append(uk)
    X.append(xk)
opti_MS.subject_to(X[-1] == 3)

U = vcat(U)
X = vcat(X)

In [24]:
opti_MS.minimize(sumsqr(U) + sumsqr(X))

opti_MS.solver('ipopt')
sol_MS = opti_MS.solve()

Usol = sol_MS.value(U) # should be [-2.7038;-0.5430;0.2613;0.5840]
print(Usol)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       14
Number of nonzeros in inequality constraint Jacobian.:        3
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:        9
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        6
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        3
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 3.00e+00 3.33e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

***In fact the ordering didn't affected the convergence. (IPOPT re-orders it for you anyway).***

In [25]:
assert_array_almost_equal(sol_MS.value(U), [-2.7038, -0.5430, 0.2613, 0.5840], decimal=4)

In [26]:
constraint_J_MS = jacobian(opti_MS.g, opti_MS.x)
constraint_J_MS.shape

(9, 9)

In [27]:
constraint_J_MS.sparsity().spy()

*........
***......
..*......
..***....
....*....
....***..
......*..
......***
........*


This is closer to the diagonal

In [28]:
H_MS = hessian(opti_MS.f, opti_MS.x)[0]
H_MS.shape

(9, 9)

In [29]:
H_MS.sparsity().spy()

*........
.*.......
..*......
...*.....
....*....
.....*...
......*..
.......*.
........*


## 5. Which transcription scales best for large N?

When working with Newton-based solvers we are going to invert or factorize KKT systems, which consist of the hessian and constraint Jacobian:

\begin{aligned}
\left[
\begin{matrix}
\nabla^2 \mathcal{L} & \frac{\partial g}{\partial x}^\mathsf{T} \\
\frac{\partial g}{\partial x}^\mathsf{T} & 0
\end{matrix}
\right]
\end{aligned}

Depending on sparsity of these components the factorization may be very cheap or very expensive.

Inverting a dense system of shape $N$-by-$N$ costs $O(N^3)$.  So if the Hessian and/or constraint Jacobian are dense, this gets very expensive for large $N$ (e.g. during system identification).

Solvers can take advantage of the sparsity of the Hessian in multiple-shooting (MS) and so it scales better to long time horizons (i.e. large $N$).

So in multiple shooting you can get a linear scaling of the complexity: $O(N)$, if the variables are well ordered (repeating small blocks along the diagonal).

## 6. Numerical values of the constraint Jacobian at initialization

In [30]:
sol_MS.value(constraint_J_MS, opti_MS.initial()).toarray()

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [31]:
sol_SS.value(constraint_J_SS, opti_SS.initial()).toarray()

array([[1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00],
       [8.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00],
       [2.56000e+02, 3.20000e+01, 1.00000e+00, 0.00000e+00],
       [1.31072e+05, 1.63840e+04, 5.12000e+02, 1.00000e+00]])

For single-shooting (SS) the initial values of the states can get high (especially if for example, the system is unstable), therefore initialization can be tricky unless you already have a reasonable intitial solution.

In multiple-shooting the simulation is restarted at each step so the system is less likely to 'blow-up'. (From a mathematical perspective, system is better 'conditioned').

## 7. Construct the KKT matrix

### Multiple shooting

In [32]:
lag_MS = opti_MS.f + opti_MS.lam_g.T @ opti_MS.g
lag_MS.shape

(1, 1)

In [33]:
G_MS = jacobian(opti_MS.g, opti_MS.x)
G_MS.shape

(9, 9)

In [34]:
DM(opti_MS.ng, opti_MS.ng)

DM(
[[00, 00, 00, 00, 00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00, 00, 00, 00, 00]])

In [35]:
KKT_MS = blockcat([
    [hessian(lag_MS, opti_MS.x)[0], G_MS.T],
    [G_MS.T, DM(opti_MS.ng, opti_MS.ng)]
])
KKT_MS.shape

(18, 18)

In [36]:
KKT_MS_t0 = sol_MS.value(KKT_MS, opti_MS.initial())
KKT_MS_t0

<18x18 sparse matrix of type '<class 'numpy.float64'>'
	with 43 stored elements in Compressed Sparse Column format>

In [37]:
(KKT_MS_t0 != 0.).toarray().astype(int)

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [38]:
np.linalg.cond(KKT_MS_t0.toarray())

15.273975133310701

### Single shooting

In [39]:
lag_SS = opti_SS.f + opti_SS.lam_g.T @ opti_SS.g
lag_SS.shape

(1, 1)

In [40]:
G_SS = jacobian(opti_SS.g,opti_SS.x)
G_SS.shape

(4, 4)

In [41]:
KKT_SS = blockcat([[hessian(lag_SS,opti_SS.x)[0],G_SS.T],[G_SS,DM(opti_SS.ng,opti_SS.ng)]])
KKT_SS.shape

(8, 8)

In [42]:
KKT_SS_t0 = sol_SS.value(KKT_SS, opti_SS.initial())
KKT_SS_t0

<8x8 sparse matrix of type '<class 'numpy.float64'>'
	with 36 stored elements in Compressed Sparse Column format>

In [43]:
(KKT_SS_t0 != 0.).toarray().astype(int)

array([[1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 0, 0, 1, 1],
       [1, 1, 1, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0, 0, 0]])

In [44]:
np.log10(KKT_SS_t0.toarray() + 1e-15).round(1)

array([[ 10.8,   9.9,   8.3,   5.4,   0. ,   0.9,   2.4,   5.1],
       [  9.9,   9. ,   7.4,   4.5, -15. ,   0. ,   1.5,   4.2],
       [  8.3,   7.4,   5.9,   3. , -15. , -15. ,   0. ,   2.7],
       [  5.4,   4.5,   3. ,   0.6, -15. , -15. , -15. ,   0. ],
       [  0. , -15. , -15. , -15. , -15. , -15. , -15. , -15. ],
       [  0.9,   0. , -15. , -15. , -15. , -15. , -15. , -15. ],
       [  2.4,   1.5,   0. , -15. , -15. , -15. , -15. , -15. ],
       [  5.1,   4.2,   2.7,   0. , -15. , -15. , -15. , -15. ]])

This matrix has very small and very large numbers--i.e. poorly condition

In [45]:
np.linalg.cond(KKT_SS_t0.toarray())

2.8047684443687567e+20

## 8. Compare the number of iterations

In [46]:
sol_SS.stats()['iter_count']

76

In [47]:
sol_MS.stats()['iter_count']

12

Order of polynomial $F(x, 0)$ is 1 (linear)

Order of polynomial $F(F(x, 0), 0)$ is 2 (quadratic)

... etc.

Therefore as the prediction horizon $N$ increases the order of the resulting polynomial gets higher.

Since each iteration involves a linearization, which is an approximation, the non-linearities will reduce the speed of convergence, requiring more iterations to converge to the solution.

## 9. Use of a reasonable initial guess

Gaving an initial guess which is close (in some sense) to the solution helps because the initial value of the objective function will be lower and therefore less iterations will be needed to reach the solution.

E.g. for system identification, you can use the input-data you have as an initial solution.

## 10. Multiple shooting with a 2x3 system

Linear system:

In [48]:
A = sparsify(DM([[1, 0.1, 0.2], [2, 0.3, 0.4], [6, 1, 3]]))
A

DM(
[[1, 0.1, 0.2], 
 [2, 0.3, 0.4], 
 [6, 1, 3]])

In [49]:
B = sparsify(DM([[1, 0], [0, 1], [2, 1]]))
# or B = sparsify(blockcat([[1,0],[0,1],[2,1]]))
B

DM(
[[1, 00], 
 [00, 1], 
 [2, 1]])

In [50]:
xk = MX.sym('xk', 3)
uk = MX.sym('uk', 2)
F = Function('F', [xk, uk], [A @ xk + B @ uk], ['xk', 'uk'], ['xkp1'])
# or simply, F = lambda x,u: A @ x + B @ u
F

Function(F:(xk[3],uk[2])->(xkp1[3]) MXFunction)

In [51]:
opti_MS = Opti()
N = 4
X = opti_MS.variable(N + 1, 3)
U = opti_MS.variable(N, 2)

# Define objective function
f = sumsqr(X) + sumsqr(U)

# Define constraints
for k in range(1, N + 1):
    xkm1, ukm1 = X[k-1, :].T, U[k-1, :].T
    opti_MS.subject_to(X[k, :] == F(xkm1, ukm1).T)
opti_MS.subject_to(X[0, :] == DM([1, 2, 3]).T)
opti_MS.subject_to(X[-1, :] == DM.zeros(3).T)

opti_MS.minimize(f)

In [52]:
opti_MS.solver('ipopt')
sol = opti_MS.solve()

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       70
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       23

Total number of variables............................:       23
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       18
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 3.00e+00 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [53]:
sol.value(U[0, :])

array([-4.28944779, -4.19298955])

In [54]:
assert_array_almost_equal(sol.value(U[0, :]), [-4.2894, -4.1930], decimal=4)

In this case (linear system, linear constraints, quadratic), single-shooting would be fine, it would converge fast (1 step).

## 11. System with 100 states

Problem:
 - System has 100 states
 - 1 control action
 - Stable dynamics
 - $\frac{\partial{F}}{\partial{x}}$ is dense
 - No path constraints
 - $N = 10000$

For single-shooting, the KKT matrix would be 10,000 blocks of size (100x100) on the diagonal (i.e. 1,000,000 x 1,000,000)!

Multiple-shooting is best because $N$ is high and dynamics are complex.  However, path constraint means that there will be 10,000 additional constraints.